---
title: "From Coach to Old Navy: a Study on the Effects of Economic Events on Luxury vs. Mass-Market Sales"
subtitle: "Spring 2025"
author: "Janelle Calalang"
bibliography: references.bib
nocite: |
  @*
number-sections: false
format:
  html:
    theme: default
    rendering: embed-resources
    code-fold: true
    code-tools: true
    toc: true
jupyter: python3
---


![Source: Unsplash.com](vasilis-caravitis-KhWNxsFWU84-unsplash.jpg){fig-alt="Shoppers walking through a mall with glass storefronts." width=80%}

# Abstract

What do elections, inflation, and luxury handbags have in common? More than you'd think. This study explores how political and economic events between 2012 and 2024 affected consumer behavior and brand value in both luxury and mass-market fashion sectors.

# The Data

Like any good design process, this started with cutting, trimming, and reshaping.

To understand how consumers responded to economic and political shifts, I gathered data from a variety of sources. Sales figures were collected from major fashion brands including Coach, Kate Spade, Stuart Weitzman, Michael Kors, H&M, The Gap, Abercrombie, and American Eagle. I also analyzed goodwill and intangible asset values using annual financial reports to track how brand equity held up over time. Since many of these reports were designed with investors in mind, they often prioritized visual appeal over data accessibility. Their layout choices (such as stylized tables, embedded text, and inconsistent formatting), made web scraping impractical. As a result, I had to manually review each report and extract the relevant figures one by one. While time-consuming, this approach ensured that the data used in my analysis was both accurate and complete. Consumer sentiment was collected from Reddit using the Pushshift API, and economic indicators like CPI and GDP were sourced from the FRED database. I also used Google Trends to trace shifts in public interest and brand engagement during periods of uncertainty.


# Exploratory Data Analysis
This phase of the project focused on identifying key patterns and preparing the data for deeper interpretation. I began by examining annual trends in brand sales, goodwill, and intangible asset values across both luxury and mass-market brands. I paired these business indicators with external factors like election years, economic volatility, and major global events such as COVID-19 to uncover potential relationships.

To supplement the financial data, I layered in public interest metrics from Google Trends and Reddit sentiment scores. I also tested for correlations between consumer engagement and brand performance. 

Because in fashion and in data, the details always matter.

## Fashion by the Figures 
To understand how consumer behavior changed across economic and political cycles, I relied on several key visualizations:

* Sales Trends by Brand (2012–2024): A line graph showing net sales across eight major fashion brands, highlighting how different labels performed over time.

* Sales + Goodwill & Intangibles Over Time: These layered line plots tracked luxury vs. mass-market brand sales alongside their goodwill and intangible asset valuations, offering insight into how brand equity held up during major events.

*  COVID-19 Sales Impact Bar Chart: A side-by-side comparison of percentage sales change from Q4 2019 to Q2 2020, showing which brands were hardest hit during the pandemic.

* Correlation Tables for Google Trends & Reddit Sentiment: These tables summarized the strength and significance of the relationship between public interest or consumer sentiment and brand sales.

* CPI, GDP & Sales by Election vs. Non-Election Quarters (Boxplots): These visuals helped compare economic indicators and sales performance during politically sensitive periods.

* Luxury vs. Mass-Market Sales & Inflation (Dual Axis Line Plot): This combined plot shows how CPI moved in relation to luxury and mass-market sales, and how each sector responded to inflation spikes.

* Word Clouds from Reddit (Mass-Market vs. Luxury): These highlighted the most commonly used descriptive words in Reddit posts for each brand category, offering a snapshot of public sentiment and product perception.

### Threading the Timeline: Sales by Brand (2012–2024)
This line graph shows yearly sales data for each brand included in the study. Mass-market brands like H&M and Gap clearly lead in overall volume, but the lines also reveal volatility, especially around 2020. In contrast, luxury brands like Coach and Michael Kors remain more stable over time, hinting at stronger consumer loyalty.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_paths = {
    "goodwill": "/Users/janellemarie/capstone/data/goodwill_value.csv",
    "yearly_sales": "/Users/janellemarie/capstone/data/yearly_sales.csv",
    "quarterly_sales": "/Users/janellemarie/capstone/data/quarterly_sales.csv",
    "h&m_goodwill": "/Users/janellemarie/capstone/data/hm_goodwill_usd.csv",
    "h&m_quarterly_sales": "/Users/janellemarie/capstone/data/hm_quarterly_sales_converted.csv",
    "h&m_sales": "/Users/janellemarie/capstone/data/hm_sales_converted.csv",
}

dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

dataframes["h&m_quarterly_sales"].rename(columns={"Net Sales (USD Millions)": "Net Sales ($ Millions)"}, inplace=True)
dataframes["h&m_goodwill"].rename(columns={"Goodwill Value (USD Millions)": "Goodwill Value ($ Millions)"}, inplace=True)

h_m_quarterly_sales = dataframes["h&m_quarterly_sales"][["Year", "Brand", "Luxury or Mass-Market", "Net Sales ($ Millions)", "Fiscal Quarter"]]
quarterly_sales = dataframes["quarterly_sales"][["Year", "Brand", "Luxury or Mass-Market", "Net Sales ($ Millions)", "Fiscal Quarter"]]
combined_quarterly_sales = pd.concat([quarterly_sales, h_m_quarterly_sales], ignore_index=True)

combined_quarterly_sales["Net Sales ($ Millions)"] = (
    combined_quarterly_sales["Net Sales ($ Millions)"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

quarterly_sales_grouped = combined_quarterly_sales.groupby(["Year", "Luxury or Mass-Market"])["Net Sales ($ Millions)"].sum().unstack()

h_m_goodwill = dataframes["h&m_goodwill"][["Year", "Brand", "Luxury or Mass-Market", "Goodwill Value ($ Millions)"]]
goodwill = dataframes["goodwill"][["Year", "Brand", "Luxury or Mass-Market", "Goodwill Value ($ Millions)"]]
combined_goodwill = pd.concat([goodwill, h_m_goodwill], ignore_index=True)

quarterly_sales = dataframes["quarterly_sales"]
h_m_quarterly_sales = dataframes["h&m_quarterly_sales"]

h_m_quarterly_sales = h_m_quarterly_sales.rename(columns={"Net Sales (USD Millions)": "Net Sales ($ Millions)"})
combined_sales = pd.concat([quarterly_sales, h_m_quarterly_sales], ignore_index=True)

combined_sales["Net Sales ($ Millions)"] = (
    combined_sales["Net Sales ($ Millions)"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

sales_trends = combined_sales.groupby("Year")["Net Sales ($ Millions)"].sum().reset_index()

combined_sales["Brand"] = combined_sales["Brand"].str.strip()

combined_sales["Brand"] = combined_sales["Brand"].replace("Gap Inc.", "Gap")

sales_trends_by_brand = combined_sales.groupby(["Year", "Brand"])["Net Sales ($ Millions)"].sum().unstack()

combined_sales.groupby(["Year", "Brand"])["Net Sales ($ Millions)"].sum().unstack()

plt.figure(figsize=(12, 6))

for brand in sales_trends_by_brand.columns:
    plt.plot(sales_trends_by_brand.index, sales_trends_by_brand[brand], marker="o", linestyle="-", label=brand)

plt.xlabel("Year")
plt.ylabel("Net Sales ($M)")
plt.title("Sales Trends by Brand (2012-2024)")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.grid(True)

# Show the plot
plt.show()

Here, I charted the goodwill values reported in brand financial statements. Despite shifts in sales, brands like Coach and Michael Kors maintained consistently high goodwill. This suggests that even during moments of uncertainty, these brands retained intangible value. Mass-market brands, on the other hand, reported minimal goodwill.


In [ ]:
combined_goodwill["Goodwill Value ($ Millions)"] = (
    combined_goodwill["Goodwill Value ($ Millions)"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

goodwill_grouped = combined_goodwill.groupby(["Year", "Luxury or Mass-Market"])["Goodwill Value ($ Millions)"].sum().unstack()

plt.figure(figsize=(12, 6))

plt.plot(quarterly_sales_grouped.index, quarterly_sales_grouped["Luxury"], label="Luxury Sales ($M)", linestyle="-", marker="o")
plt.plot(quarterly_sales_grouped.index, quarterly_sales_grouped["Mass-Market"], label="Mass-Market Sales ($M)", linestyle="--", marker="o")

plt.plot(goodwill_grouped.index, goodwill_grouped["Luxury"], label="Luxury Goodwill ($M)", linestyle="-", marker="s")
plt.plot(goodwill_grouped.index, goodwill_grouped["Mass-Market"], label="Mass-Market Goodwill ($M)", linestyle="--", marker="s")

events = {
    2012: "2012 Election",
    2016: "2016 Election",
    2020: "COVID-19 & 2020 Election",
    2022: "Inflation Surge",
    2024: "2024 Election"
}
for year, label in events.items():
    plt.axvline(x=year, color="gray", linestyle=":", alpha=0.7)
    plt.text(year, 54000, label, rotation=25, ha="center", va="top", fontsize=9)

plt.xlabel("Year")
plt.ylabel("Sales & Goodwill ($M)")
plt.title("Sales Trends and Goodwill Valuations (2012–2024)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

This visualization focuses on intangible assets beyond goodwill, like brand identity, customer relationships, and intellectual property. Once again, luxury brands dominate. The numbers reinforce the idea that luxury is built on more than product. It’s about identity, story, and legacy.


In [ ]:
goodwill = pd.read_csv("/Users/janellemarie/datasets/goodwill_value.csv")
hm_goodwill = pd.read_csv("/Users/janellemarie/datasets/hm_goodwill_usd.csv")

combined_intangibles = pd.concat([goodwill, hm_goodwill], ignore_index=True)

combined_intangibles["Intangible Assets ($ Millions)"] = (
    combined_intangibles["Intangible Assets ($ Millions)"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

intangibles_grouped = combined_intangibles.groupby(["Year", "Luxury or Mass-Market"])["Intangible Assets ($ Millions)"].sum().unstack()
intangibles_grouped.head()

plt.figure(figsize=(12, 6))

plt.plot(quarterly_sales_grouped.index, quarterly_sales_grouped["Luxury"], label="Luxury Sales ($M)", linestyle="-", marker="o")
plt.plot(quarterly_sales_grouped.index, quarterly_sales_grouped["Mass-Market"], label="Mass-Market Sales ($M)", linestyle="--", marker="o")

plt.plot(intangibles_grouped.index, intangibles_grouped["Luxury"], label="Luxury Intangibles ($M)", linestyle="-", marker="s")
plt.plot(intangibles_grouped.index, intangibles_grouped["Mass-Market"], label="Mass-Market Intangibles ($M)", linestyle="--", marker="s")

events = {
    2012: "2012 Election",
    2016: "2016 Election",
    2020: "COVID-19 & 2020 Election",
    2022: "Inflation Surge",
    2024: "2024 Election"
}

for year, label in events.items():
    plt.axvline(x=year, color="gray", linestyle=":", alpha=0.7)
    plt.text(year, 54000, label, rotation=25, ha="center", va="top", fontsize=9)

plt.xlabel("Year")
plt.ylabel("Sales & Intangible Assets ($M)")
plt.title("Sales Trends and Intangible Asset Valuations (2012–2024)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### When the Malls Closed: COVID-19 Sales Impact
![COVID-era fashion consumer](arturo-rey-5yP83RhaFGA-unsplash.jpg)

In this bar chart, I isolated the difference in sales between Q4 2019 and Q2 2020 to capture the immediate impact of COVID-19. H&M saw the sharpest decline, while brands like Coach and Stuart Weitzman actually experienced growth. This highlights the resilience of certain luxury brands in contrast to the vulnerability of mass-market players during crisis moments.


In [ ]:
sales_2019_q4 = combined_sales[(combined_sales["Year"] == 2019) & (combined_sales["Fiscal Quarter"] == "Q4")].groupby("Brand")["Net Sales ($ Millions)"].sum()
sales_2020_q2 = combined_sales[(combined_sales["Year"] == 2020) & (combined_sales["Fiscal Quarter"] == "Q2")].groupby("Brand")["Net Sales ($ Millions)"].sum()

sales_change_2020 = pd.DataFrame({"Q4 2019 Sales": sales_2019_q4, "Q2 2020 Sales": sales_2020_q2})
sales_change_2020.fillna(0, inplace=True)
sales_change_2020["Percentage Change"] = ((sales_change_2020["Q2 2020 Sales"] - sales_change_2020["Q4 2019 Sales"]) / 
sales_change_2020["Q4 2019 Sales"].replace(0, 1)) * 100
sales_change_2020_sorted = sales_change_2020.sort_values(by="Percentage Change")

colors_by_change = sales_change_2020_sorted["Percentage Change"].apply(lambda x: "teal" if x < 0 else "gold")

plt.figure(figsize=(12, 8))
plt.barh(sales_change_2020_sorted.index, sales_change_2020_sorted["Percentage Change"], color=colors_by_change)
plt.xlabel("Percentage Change in Sales (%)")
plt.ylabel("Brand")
plt.title("Sales Change from Q4 2019 to Q2 2020 (COVID-19 Impact)")
plt.axvline(x=0, color="black", linewidth=1)
plt.show()

### Google Search Interest Correlation Analysis
This is a correlation test between Google Trends search interest against brand sales. Abercrombie and H&M show a strong positive correlation, suggesting that curiosity and conversion were linked. Interestingly, American Eagle breaks that trend—more search interest actually aligned with fewer sales, which may reflect negative buzz or unmet consumer expectations.


In [ ]:
import pandas as pd
from scipy.stats import pearsonr

path = "/Users/janellemarie/datasets/"

sales = pd.read_csv(path + "quarterly_sales.csv")
hm_sales = pd.read_csv(path + "hm_quarterly_sales_converted.csv")
hm_sales = hm_sales.rename(columns={"Net Sales (USD Millions)": "Net Sales ($ Millions)"})

combined = pd.concat([sales, hm_sales], ignore_index=True)
combined["Net Sales ($ Millions)"] = combined["Net Sales ($ Millions)"].astype(str).str.replace(",", "").astype(float)
combined["Brand"] = combined["Brand"].str.strip().replace("Gap Inc.", "Gap")

quarter_map = {"Q1": "01", "Q2": "04", "Q3": "07", "Q4": "10"}
combined["Month"] = combined["Fiscal Quarter"].map(quarter_map)
combined["Date"] = pd.to_datetime(combined["Year"].astype(str) + "-" + combined["Month"] + "-01")

monthly_sales = combined.groupby(["Date", "Brand"])["Net Sales ($ Millions)"].sum().reset_index()

files = [
    "abercrombie_search.csv",
    "americaneagle_search.csv",
    "coach_search.csv",
    "gap_search.csv",
    "hm_search.csv",
    "katespade_search.csv",
    "michaelkors_search.csv",
    "stuart_search.csv"
]

for file in files:
    brand_name = file.replace("_search.csv", "")
    if brand_name == "americaneagle":
        brand_name = "American Eagle"
    elif brand_name == "katespade":
        brand_name = "Kate Spade"
    elif brand_name == "michaelkors":
        brand_name = "Michael Kors"
    elif brand_name == "stuart":
        brand_name = "Stuart Weitzman"
    elif brand_name == "hm":
        brand_name = "H&M"
    else:
        brand_name = brand_name.capitalize()

    search = pd.read_csv(path + file)
    search["Date"] = pd.to_datetime(search["Date"])
    search["Search Interest"] = pd.to_numeric(search["Search Interest"], errors="coerce")
    search["Brand"] = brand_name

    merged = pd.merge(search, monthly_sales, on=["Date", "Brand"], how="inner")

    if len(merged) >= 2:
        corr, pval = pearsonr(merged["Search Interest"], merged["Net Sales ($ Millions)"])
        print(f"{brand_name}: Correlation = {round(corr, 3)} | P-Value = {round(pval, 4)}")

### Reddit Consumer Sentiment Correlation Analysis
H&M and American Eagle showed positive relationships between sentiment and sales, while Gap trailed behind. This suggests that positive community perception, especially on platforms like Reddit, may have some correlation to support stronger performance.


In [ ]:
yearly_sales = combined.groupby(["Year", "Brand"])["Net Sales ($ Millions)"].sum().reset_index()
yearly_sales["Sales Change (%)"] = yearly_sales.groupby("Brand")["Net Sales ($ Millions)"].pct_change() * 100

sentiment = pd.read_csv(path + "reddit_sentiment_by_year.csv")

merged = pd.merge(sentiment, yearly_sales, on=["Year", "Brand"], how="inner").dropna()

brands = merged["Brand"].unique()
for brand in brands:
    df = merged[merged["Brand"] == brand]
    if len(df) >= 2:
        corr, pval = pearsonr(df["Avg Sentiment"], df["Sales Change (%)"])
        print(f"{brand}: Correlation = {round(corr, 3)} | P-Value = {round(pval, 4)}")

### Election Years on the Runway
This triple boxplot compares GDP, CPI, and sales across pre-election, election, and post-election quarters. While GDP and sales don’t shift dramatically, CPI tends to dip slightly during election quarters, possibly reflecting political efforts to stabilize consumer prices. This graph displays fashion consumption in a broader macroeconomic context.


In [ ]:
import seaborn as sns
gdp = pd.read_csv("/Users/janellemarie/datasets/GDP_growth.csv")
cpi = pd.read_csv("/Users/janellemarie/datasets/MEDCPI.csv")

gdp = gdp.rename(columns={"observation_date": "Date", "A191RL1Q225SBEA": "GDP"})
gdp["Quarter"] = pd.to_datetime(gdp["Date"]).dt.to_period("Q")

cpi = cpi.rename(columns={"observation_date": "Date", "MEDCPIM158SFRBCLE": "CPI"})
cpi["Quarter"] = pd.to_datetime(cpi["Date"]).dt.to_period("Q")
cpi = cpi.groupby("Quarter")["CPI"].mean().reset_index()

merged = quarterly_sales.copy()
merged.rename(columns={"Fiscal Quarter": "Quarter"}, inplace=True)
merged["Quarter"] = merged["Year"].astype(str) + merged["Quarter"]
merged["Quarter"] = pd.PeriodIndex(merged["Quarter"], freq="Q")

merged = merged.merge(gdp[["Quarter", "GDP"]], on="Quarter", how="left")
merged = merged.merge(cpi, on="Quarter", how="left")

election_quarters = ['2012Q4', '2016Q4', '2020Q4', '2024Q4']
merged['Election_Quarter'] = merged['Quarter'].astype(str).isin(election_quarters)

merged['Net Sales ($ Millions)'] = pd.to_numeric(merged['Net Sales ($ Millions)'], errors='coerce')


sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True)

sns.boxplot(x='Election_Quarter', y='GDP', data=merged, ax=axes[0])
axes[0].set_title("GDP by Election vs. Non-Election Quarters")
axes[0].set_xlabel("")
axes[0].set_ylabel("GDP (Billions)")

sns.boxplot(x='Election_Quarter', y='CPI', data=merged, ax=axes[1])
axes[1].set_title("CPI by Election vs. Non-Election Quarters")
axes[1].set_xlabel("")
axes[1].set_ylabel("Median CPI")

sns.boxplot(x='Election_Quarter', y='Net Sales ($ Millions)', data=merged, ax=axes[2])
axes[2].set_title("Sales by Election vs. Non-Election Quarters")
axes[2].set_xlabel("")
axes[2].set_ylabel("Net Sales ($M)")

for ax in axes:
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Non-Election', 'Election'])


plt.tight_layout()
plt.show()

This dual-axis plot tracks the relationship between luxury and mass-market sales alongside inflation (measured by median CPI) from 2012 to 2024. Mass-market brands remained relatively steady in sales, with a notable dip during the 2020 election and pandemic before bouncing back. Luxury sales, while more volatile, show steady growth until 2022, when rising inflation appears to coincide with a decline. The pattern suggests that inflation might affect luxury consumers differently. Still, both sectors held their ground surprisingly well. I guess price tags don't sway easily under economic pressure. 


In [ ]:
cpi = pd.read_csv("/Users/janellemarie/datasets/MEDCPI.csv")
cpi["Date"] = pd.to_datetime(cpi["observation_date"])
cpi = cpi.rename(columns={"MEDCPIM158SFRBCLE": "Median CPI"})
cpi_yearly = cpi.set_index("Date")["Median CPI"].resample("Y").mean().reset_index()
cpi_yearly["Year"] = cpi_yearly["Date"].dt.year
cpi_yearly = cpi_yearly[["Year", "Median CPI"]]

sales = pd.read_csv("/Users/janellemarie/datasets/yearly_sales.csv")

if "Year" not in sales.columns:
    sales["Year"] = pd.to_datetime(sales["Date"]).dt.year
else:
    sales["Year"] = sales["Year"].astype(int)

sales_summary = sales.groupby(["Year", "Luxury or Mass-Market"])["Net Sales ($ Millions)"].sum().unstack().reset_index()

merged = pd.merge(sales_summary, cpi_yearly, on="Year", how="inner").rename(columns={"Median CPI": "CPI"})
merged = merged[merged["Year"].between(2012, 2024)]

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(merged["Year"], merged["Luxury"], color='orange', marker='o', label='Luxury Sales ($M)')
ax1.plot(merged["Year"], merged["Mass-Market"], color='orangered', marker='o', linestyle='--', label='Mass-Market Sales ($M)')
ax1.set_ylabel("Sales ($M)", color='darkred')
ax1.tick_params(axis='y', labelcolor='darkred')
ax1.set_xlabel("Year")
ax1.set_title("Luxury vs. Mass-Market Sales & Inflation")

ax2 = ax1.twinx()
ax2.plot(merged["Year"], merged["CPI"], color='gray', linestyle='-.', marker='x', label='Median CPI')
ax2.set_ylabel("Median CPI", color='gray')
ax2.tick_params(axis='y', labelcolor='gray')

events = {2012: "2012 Election", 2016: "2016 Election", 2020: "2020 Election\n& COVID-19", 2022: "Inflation Surge", 2024: "2024 Election"}
for year, label in events.items():
    ax1.axvline(x=year, color='gray', linestyle='--', linewidth=1)
    ax1.annotate(label, xy=(year, ax1.get_ylim()[1] * 0.85), rotation=45, ha='right', va='center', fontsize=8, color='black')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3)

plt.tight_layout()
plt.show()

### Reddit, but Make It Fashion
Finally, I created two word clouds from Reddit data to capture the tone of consumer conversations. Together, these visuals show how people talk about fashion very differently depending on the brand category.


In [ ]:
## tutorial source: https://medium.com/mcd-unison/create-word-cloud-scraping-data-from-reddit-api-using-praw-and-spacy-b5c9c61c2d10
import praw
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

reddit = praw.Reddit(
    client_id="8DsIW4DAuPnHNFSqaFj98A",
    client_secret="FkL4hqDJc81s5JSo02eoB65eqDCWsw",
    user_agent="CapstoneSentimentApp by janellemariie"
)

mass_market_brands = {
    "Abercrombie": "abercrombie fashion",
    "American Eagle": "american eagle outfit",
    "Gap": "gap clothing",
    "H&M": "hm haul"
}

luxury_brands = {
    "Coach": "coach bag",
    "Kate Spade": "kate spade purse",
    "Michael Kors": "michael kors bag",
    "Stuart Weitzman": "stuart weitzman boots"
}

subreddits = ["femalefashionadvice", "malefashionadvice", "frugalmalefashion", "trendygirl", "streetwear"]

def fetch_text(keywords_dict):
    combined = ""
    for brand, query in keywords_dict.items():
        for sub in subreddits:
            try:
                for post in reddit.subreddit(sub).search(query, sort="relevance", limit=75):
                    if post.selftext and len(post.selftext.split()) > 20:
                        combined += post.title + " " + post.selftext + " "
                    elif len(post.title.split()) > 5:
                        combined += post.title + " "
            except:
                continue
    return combined

mass_text = fetch_text(mass_market_brands)
luxury_text = fetch_text(luxury_brands)

descriptive_keywords = {
    "cute", "great", "nice", "soft", "quality", "durable", "affordable", "expensive", "trendy",
    "versatile", "timeless", "cozy", "stylish", "classic", "reliable", "comfy", "flattering",
    "minimal", "bold", "structured", "cheap", "worth", "iconic", "modern", "vintage", "beautiful",
    "elegant", "perfect", "simple", "neutral", "amazing", "chic", "feminine", "masculine", "luxurious"
}

def keep_only_keywords(text, keyword_set):
    words = text.lower().split()
    return " ".join([word.strip(".,!?()[]\"'") for word in words if word.strip(".,!?()[]\"'") in keyword_set])

mass_filtered = keep_only_keywords(mass_text, descriptive_keywords)
luxury_filtered = keep_only_keywords(luxury_text, descriptive_keywords)

mass_wc = WordCloud(width=1000, height=500, background_color="white").generate(mass_filtered)
luxury_wc = WordCloud(width=1000, height=500, background_color="white").generate(luxury_filtered)

plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
plt.imshow(mass_wc, interpolation='bilinear')
plt.axis('off')
plt.title("Mass-Market Descriptive Discourse")

plt.subplot(1, 2, 2)
plt.imshow(luxury_wc, interpolation='bilinear')
plt.axis('off')
plt.title("Luxury Brand Descriptive Discourse")

plt.tight_layout()
plt.show()

# Conclusion
This project began with a simple question: does the economy influence where people choose to shop? After examining over a decade of brand data alongside economic indicators, election cycles, and online sentiment, the answer reveals itself in patterns that are subtle but consistent. Luxury and mass-market brands do not respond to financial pressure in the same way. Mass-market labels often depend on volume and price sensitivity, while luxury brands depend on long-term brand equity and emotional loyalty from consumers.

Events like the 2020 election and the onset of the COVID-19 pandemic were not just sales disruptors. They also marked shifts in how people searched for and discussed fashion online. Public interest data from Google and sentiment pulled from Reddit posts added dimension to the financial trends.

Fashion has always reflected more than personal taste. It reflects timing, perception, and the economic realities people live in. Understanding those relationships can help brands anticipate how style survives uncertainty.

## Final Thoughts
I went into this project with little confidence. I chose a topic I cared deeply about, knowing it would involve more manual work and messier data than most. The data cleaning process was frustrating at times and pushed me to the edge of burnout, but in the end, I’m glad I followed my instincts. This project reminded me that research feels more meaningful when it’s rooted in something personal to me.

Looking ahead, I hope to expand this work by exploring the fast fashion boom in the years following COVID-19. I’m especially interested in how sustainability efforts have evolved during this time and what that means for consumers, brands, and the planet. There’s still so much to uncover in the space where economics and clothing intersect.